<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/twadr_2_pubmed_bigtweet_mixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE html>'
100  9115  100  9115    0     0  28219      0 --:--:-- --:--:-- --:--:-- 28132
curl: (23) Failed writing body (290 != 1213)


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'TwADR-L.fold-2.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-2.validation.csv",header=None)
valid.head()

,0,1
0,13,insomnia
1,24,heart beat for being too strong
2,62,feeling proper psycho
3,94,anxiety
4,133,night terrors


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-2.test.csv",header=None)
test.head()

,0,1
0,13,It's 6 am and this adderall got me reevaluatin...
1,24,racing heart
2,56,persistent painful erection
3,57,hypertrophic scarring
4,94,anxiety


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.231883,4.640844,0.249684,15:44


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.482818,4.291632,0.283770,15:44


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.165046,4.251271,0.288207,15:49


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.102047,4.221434,0.291211,15:40


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.158709,6.690338,0.057143,00:23


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.250553,5.750672,0.078571,00:27


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.765199,5.297599,0.121429,01:04


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.520971,4.920454,0.200000,01:08
1,5.353920,4.798950,0.192857,01:07


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.142678,4.718239,0.164286,01:03
1,4.972838,4.597039,0.192857,01:04


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.822379,4.439468,0.214286,01:08
1,4.682860,4.401518,0.200000,01:03


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.483736,4.336553,0.192857,01:03
1,4.385328,4.318506,0.235714,01:06


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.195281,4.340152,0.228571,01:06
1,4.099548,4.300251,0.228571,01:02


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.937778,4.315047,0.235714,01:07
1,3.891862,4.213424,0.250000,01:08
2,3.727410,4.122868,0.235714,01:07
3,3.602588,4.139079,0.214286,01:04


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.474163,4.152463,0.228571,01:06
1,3.414977,4.251928,0.221429,01:08
2,3.246346,4.179151,0.228571,01:09
3,3.136743,4.198519,0.228571,01:04


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.979514,4.189526,0.228571,01:07
1,3.012445,4.202238,0.271429,01:03
2,2.874618,4.139605,0.264286,01:07
3,2.762772,4.131662,0.257143,01:04


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.622118,4.105123,0.264286,01:04
1,2.633357,4.256101,0.271429,01:09
2,2.538068,4.218416,0.271429,01:06
3,2.448661,4.257504,0.271429,01:07


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.291505,4.195203,0.257143,01:07
1,2.346600,4.245503,0.271429,01:05
2,2.267129,4.195296,0.271429,01:06
3,2.184495,4.276186,0.264286,01:05


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.165401,4.248738,0.278571,01:06
1,2.108050,4.348323,0.271429,01:07
2,2.099913,4.405500,0.271429,01:09
3,2.014362,4.334878,0.278571,01:07


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.865833,4.349148,0.271429,01:05
1,1.982203,4.599866,0.257143,01:07
2,1.934851,4.448007,0.264286,01:06
3,1.874580,4.498684,0.264286,01:05


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.837699,4.584334,0.264286,01:08
1,1.912012,4.571594,0.271429,01:07
2,1.895543,4.606130,0.250000,01:03
3,1.857614,4.649079,0.250000,01:10


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.763038,4.574872,0.285714,01:07
1,1.809895,4.689910,0.278571,01:08
2,1.838870,4.761073,0.278571,01:06
3,1.774006,4.734767,0.278571,01:10


In [43]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.694981,4.795897,0.278571,01:03
1,1.704177,4.777794,0.264286,01:06


In [44]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.726132,4.734345,0.264286,01:05
1,1.738074,4.715613,0.278571,01:03


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [46]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [47]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[978, 24, 1727, 1370, 978, 852, 1340, 1340, 189, 1340, 1531, 1521, 2147, 1936, 2147, 108, 301, 908, 1340, 1340, 396, 908, 445, 824, 582, 2097, 520, 720, 744, 744, 749, 1706, 749, 887, 1340, 1073, 792, 792, 2125, 1340, 824, 97, 265, 1184, 1973, 974, 974, 974, 978, 1507, 1019, 359, 359, 1340, 620, 1340, 1925, 1340, 1340, 1136, 1142, 1340, 1340, 154, 582, 908, 1265, 1340, 1295, 852, 1340, 93, 1340, 908, 1340, 1507, 908, 2191, 1295, 1973, 1340, 582, 1340, 712, 1507, 1526, 1529, 1529, 1340, 978, 1196, 1597, 1597, 1767, 1767, 1597, 1340, 108, 1340, 1701, 1196, 1340, 1145, 1463, 1099, 1099, 1340, 1099, 1099, 1340, 1768, 359, 1340, 1787, 1787, 1340, 393, 1196, 908, 1810, 1718, 1073, 750, 1936, 1196, 233, 1936, 1936, 1340, 1340, 1973, 923, 1767, 579, 750, 824, 1099, 1676, 2147, 2191, 1340, 1787, 13]
[13, 24, 56, 57, 94, 113, 189, 189, 189, 189, 217, 224, 233, 233, 233, 265, 301, 393, 393, 393, 396, 418, 445, 452, 582, 605, 609, 720, 744, 745, 749, 749, 749, 749, 755, 791, 792, 792, 813,

In [48]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

39
0.2727272727272727
